# DataDex

## Project Description

The aim of this project is to predict the outcome of a one-versus-one Pokémon battle based on the Pokémons' statistics (Attack, Defense etc), their type (Poison, Normal etc) and win-loss data from previous battles.

No Pokémon were harmed during this analysis.

In [1]:
# Imports
import pandas as pd

In [2]:
pokemon_df = pd.read_csv('Dataset\pokemon.csv')

# Renaming "#" to "Number"
pokemon_df = pokemon_df.rename(index=str, columns={"#": "Number"})

pokemon_df.head()

,Number,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


In [3]:
pokemon_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 800 entries, 0 to 799
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Number      800 non-null    int64 
 1   Name        799 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   HP          800 non-null    int64 
 5   Attack      800 non-null    int64 
 6   Defense     800 non-null    int64 
 7   Sp. Atk     800 non-null    int64 
 8   Sp. Def     800 non-null    int64 
 9   Speed       800 non-null    int64 
 10  Generation  800 non-null    int64 
 11  Legendary   800 non-null    bool  
dtypes: bool(1), int64(8), object(3)
memory usage: 75.8+ KB


## Understanding the dataset
**Number:** Refers to the Pokémon's number in the [List of Pokémon by National Pokédex number](https://bulbapedia.bulbagarden.net/wiki/List_of_Pokémon_by_National_Pokédex_number).

**Type 1:** All Pokémon have a main type. Type 1 refers to the main type.

**Type 2:** Some Pokémon have a secondary type. Type 2 refers to the secondary type.

**HP, Attack, Defense, Sp. Atk, Sp. Def, Speed:** Refers to the Pokémon's stats.

**Generation:** Refers to the season of the show during which the Pokémon was released.

**Legendary:** Special attribute that only very few Pokémon have.

In [4]:
# Empty values in the Pokémon DF
pokemon_df.isnull().sum()

Number          0
Name            1
Type 1          0
Type 2        386
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

Empty values in Type 2 is normal, since some Pokémon don't have a secondary type.
From the analysis, we can deduce that there are 386 Pokémon without a secondary type.
We have one missing Pokémon name. Let's find it using this [link](https://bulbapedia.bulbagarden.net/wiki/List_of_Pokémon_by_National_Pokédex_number).

In [5]:
print("This pokemon is before the missing Pokemon: " + pokemon_df['Name'][61])
print(pokemon_df[pokemon_df['Name'].isnull()])
print("This pokemon is after the missing Pokemon: " + pokemon_df['Name'][63])

This pokemon is before the missing Pokemon: Mankey
    Number Name    Type 1 Type 2  HP  Attack  Defense  Sp. Atk  Sp. Def  \
62      63  NaN  Fighting    NaN  65     105       60       60       70   

    Speed  Generation  Legendary  
62     95           1      False  
This pokemon is after the missing Pokemon: Growlithe


So the missing Pokémon's number is 63. Using the link, we can see that the name of the missing Pokémon is Primeape. Let's add it.

In [6]:
# pokemon_df['Name'][62] = "Primeape" <- returns SettingWithCopyWarning

pokemon_df.loc[pokemon_df.Number == 63, 'Name'] = "Primeape"

# Display the new data
primeape = pokemon_df[pokemon_df["Number"] == 63]
primeape

,Number,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
62,63,Primeape,Fighting,NaN,65,105,60,60,70,95,1,False


In [7]:
combat_df = pd.read_csv('Dataset\combats.csv')

combat_df.head()

,First_pokemon,Second_pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668
3,237,683,683
4,151,231,151
